In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mizani.formatters import percent_format
import os
from plotnine import *
import numpy as np
import sys
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse

In [3]:
# Reading the data
total_data = pd.read_csv("https://osf.io/download/4ay9x/")

# Checking if the data was correctly loaded
total_data.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


### Data manipulation
In this part, data is filtered and some additional variables are added for the modeling.

In [13]:
# filtering for civil engineers (occupation code = 1360)
data = total_data[total_data["occ2012"] == 1360]
print(f"There are {data.shape[0]} civil engineers in the dataset")

There are 396 civil engineers in the dataset


In [17]:
# checking data completeness
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 168 to 148438
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  396 non-null    int64  
 1   hhid        396 non-null    int64  
 2   intmonth    396 non-null    object 
 3   stfips      396 non-null    object 
 4   weight      396 non-null    float64
 5   earnwke     396 non-null    float64
 6   uhours      396 non-null    int64  
 7   grade92     396 non-null    int64  
 8   race        396 non-null    int64  
 9   ethnic      36 non-null     float64
 10  age         396 non-null    int64  
 11  sex         396 non-null    int64  
 12  marital     396 non-null    int64  
 13  ownchild    396 non-null    int64  
 14  chldpres    396 non-null    int64  
 15  prcitshp    396 non-null    object 
 16  state       396 non-null    object 
 17  ind02       396 non-null    object 
 18  occ2012     396 non-null    int64  
 19  class       396 non-null    o

In [23]:
# creating hourly wage, binary gender variable (True for males, False for females) and "white" (True if race = 1 - i.e. white, else 0) variables 
data["wage"] = data["earnwke"] / data["uhours"]
data["gender"] = data["sex"] == 1
data["white"] = data["race"] == 1
data

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,state,ind02,occ2012,class,unionmme,unioncov,lfsr94,wage,gender,white
168,394,4540720924693,January,AK,938.8809,615.00,40,43,4,NaN,...,94,"Architectural, engineering, and related servic...",1360,"Private, For Profit",No,No,Employed-At Work,15.375000,True,False
179,412,51250720790591,January,AK,374.8012,1250.00,38,44,1,NaN,...,94,Administration of economic programs and space ...,1360,Government - State,Yes,NaN,Employed-At Work,32.894737,True,True
697,1575,260177093001600,January,CA,3478.9719,1346.00,50,43,1,7.0,...,93,Petroleum refining (32411),1360,"Private, For Profit",No,No,Employed-At Work,26.920000,True,True
738,1679,310864092903826,January,CA,3089.1716,769.23,40,43,1,5.0,...,93,Hospitals (622),1360,"Private, For Profit",Yes,NaN,Employed-At Work,19.230750,True,True
1548,3502,972046964079070,January,CA,3371.2601,2076.92,40,43,4,NaN,...,93,Administration of economic programs and space ...,1360,Government - State,Yes,NaN,Employed-At Work,51.923000,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148115,314463,94100920950665,December,VT,263.7911,1194.00,40,40,1,NaN,...,1,** Construction (23),1360,Government - State,Yes,NaN,Employed-At Work,29.850000,True,True
148144,314533,441803003350100,December,VT,274.4192,1955.20,40,43,1,NaN,...,1,Services incidental to transportation (488),1360,Government - State,No,No,Employed-At Work,48.880000,True,True
148172,314602,710008645900880,December,VT,270.2848,1346.15,40,43,1,NaN,...,1,"Architectural, engineering, and related servic...",1360,"Private, For Profit",No,No,Employed-At Work,33.653750,True,True
148181,314621,840130301040503,December,VT,285.5574,1095.00,55,43,1,NaN,...,1,"Architectural, engineering, and related servic...",1360,"Private, For Profit",No,No,Employed-At Work,19.909091,True,True


In [24]:
# creating highest education dummies: 
# "hs" for high-school or lower education
# "ba" for associates degrees or BAs 
# "ma" for masters and professional degrees
# "phd" for PhDs

data["edu_hs"] = data["grade92"] <= 39
data["edu_ba"] = data["grade92"].isin([39, 40, 41, 42, 43])
data["edu_ma"] = data["grade92"].isin([44, 45])
data["edu_phd"] = data["grade92"] == 46
data

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,unionmme,unioncov,lfsr94,wage,gender,white,edu_hs,edu_ba,edu_ma,edu_phd
168,394,4540720924693,January,AK,938.8809,615.00,40,43,4,NaN,...,No,No,Employed-At Work,15.375000,True,False,False,True,False,False
179,412,51250720790591,January,AK,374.8012,1250.00,38,44,1,NaN,...,Yes,NaN,Employed-At Work,32.894737,True,True,False,False,True,False
697,1575,260177093001600,January,CA,3478.9719,1346.00,50,43,1,7.0,...,No,No,Employed-At Work,26.920000,True,True,False,True,False,False
738,1679,310864092903826,January,CA,3089.1716,769.23,40,43,1,5.0,...,Yes,NaN,Employed-At Work,19.230750,True,True,False,True,False,False
1548,3502,972046964079070,January,CA,3371.2601,2076.92,40,43,4,NaN,...,Yes,NaN,Employed-At Work,51.923000,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148115,314463,94100920950665,December,VT,263.7911,1194.00,40,40,1,NaN,...,Yes,NaN,Employed-At Work,29.850000,True,True,False,True,False,False
148144,314533,441803003350100,December,VT,274.4192,1955.20,40,43,1,NaN,...,No,No,Employed-At Work,48.880000,True,True,False,True,False,False
148172,314602,710008645900880,December,VT,270.2848,1346.15,40,43,1,NaN,...,No,No,Employed-At Work,33.653750,True,True,False,True,False,False
148181,314621,840130301040503,December,VT,285.5574,1095.00,55,43,1,NaN,...,No,No,Employed-At Work,19.909091,True,True,False,True,False,False


In [25]:
# creating union dummy (True for members, False for non-members)
data["union"] = data["unionmme"] == 1

In [26]:
# creating squared and cubed age variables for potential non-linear associations
data["age_2"] = data["age"] ** 2
data["age_3"] = data["age"] ** 3

## Modeling
Creating 4 OLS models with increasing complexity to predict hourly wages:
- Model 1 to include: age
- Model 2 to include: age, age squared, age cubed, gender
- Model 3 to include: age, age squared, age cubed, gender, education dummies
- Model 4 to include: age, age squared, age cubed, gender, education dummies, union membership, race

Note: for the education dummies, High-school level is the comparison, therefore it is left out of the OLS models.

In [66]:
reg1 = smf.ols("wage ~ age", data=data).fit(cov_type="HC0")
reg2 = smf.ols("wage ~ age + age_2 + age_3 + gender", data=data).fit(cov_type="HC0")
reg3 = smf.ols("wage ~ age + age_2 + age_3 + gender + edu_ba + edu_ma + edu_phd", data=data).fit(cov_type="HC0")
reg4 = smf.ols("wage ~ age + age_2 + age_3 + gender + edu_ba + edu_ma + edu_phd + union + race", data=data).fit(cov_type="HC0")

In [69]:
# comparing models and adding BIC scores

bic = [round(x.bic, 2) for x in [reg1,reg2,reg3,reg4]]
sg = stargazer.Stargazer([reg1,reg2,reg3,reg4])
sg.covariate_order(["age", "age_2", "age_3", "edu_ba[T.True]", "edu_ma[T.True]", "edu_phd[T.True]", "gender[T.True]", "race", "union[T.True]", "Intercept"])
sg.rename_covariates({"age": "Age", "age_2": "Age squared", "age_3": "Age cubed", 
                      "edu_ba[T.True]" : "BA education", "edu_ma[T.True]" : "MA education", "edu_phd[T.True]" : "PhD education", 
                      "gender[T.True]" : "Gender", 
                      "race" : "Race", 
                      "union[T.True]" : "Union membership"})
sg.add_line('BIC', bic, location=stargazer.LineLocation.FOOTER_BOTTOM)
sg

BIC scores are very similar. Although Model 3 has a slightly lower score than Model 1, the latter may still be favourable as it is a significanlty simpler model.

In [79]:
rmse1 = rmse(reg1.fittedvalues,data.wage).round(3)
rmse2 = rmse(reg2.fittedvalues,data.wage).round(3)
rmse3 = rmse(reg3.fittedvalues,data.wage).round(3)
rmse4 = rmse(reg4.fittedvalues,data.wage).round(3)

rmse_summary = pd.DataFrame(
    {"Model 1": rmse1,
    "Model 2" : rmse2,
    "Model 3" : rmse3,
    "Model 4" : rmse4}, index = ["RMSE"]
)
rmse_summary

,Model 1,Model 2,Model 3,Model 4
RMSE,15.989,15.779,15.19,15.19


Based on RMSE Model 3 and 4 perform the best. Since the scores of these two models are very similar, again the simpler model (i.e. Model 3) would be preferred.

### Cross-validated RMSE

In [80]:
from sklearn.model_selection import KFold
k = KFold(n_splits=4, shuffle=True, random_state=42)

In [81]:
for train_index, test_index in k.split(data):
    print(train_index, '\n', '\n', test_index, '\n')

[  1   2   4   6   7   8  10  11  12  13  14  16  17  19  20  21  23  24
  26  27  28  29  32  34  35  36  37  38  40  41  43  44  47  48  49  50
  51  52  53  54  58  59  60  61  62  64  65  66  67  68  69  71  74  79
  80  81  83  85  86  87  88  89  91  92  95  96  97  98  99 100 102 103
 105 106 107 109 111 112 115 117 118 119 120 121 122 123 125 126 127 128
 129 130 133 134 135 136 138 139 140 142 143 144 145 146 147 148 149 150
 151 153 154 155 156 159 160 161 162 163 164 165 166 167 169 170 171 174
 178 179 180 182 183 184 185 186 187 188 189 190 191 192 193 194 196 197
 199 200 201 202 204 205 206 207 209 210 212 213 214 215 216 217 218 219
 220 221 222 224 225 226 230 231 232 233 234 235 236 237 238 239 240 241
 242 243 244 245 246 249 251 252 253 254 256 257 259 260 263 264 265 266
 267 269 270 272 273 274 276 278 279 280 281 282 284 285 286 287 288 289
 290 291 293 294 295 297 298 299 300 301 302 303 304 306 307 308 309 310
 312 313 314 315 316 318 319 320 323 324 325 327 32

In [82]:
### Cross validate OLS with combining sklearn k-fold cross validation and statsmodels ols formula


def cv_reg(formula, data, kfold, robustse=None):
    regression_list = []
    predicts_on_test = []
    rsquared = []
    rmse_list = []

    # Calculating OLS for each fold

    for train_index, test_index in k.split(data):
        # print("TRAIN:", train_index, "TEST:", test_index)
        data_train, data_test = data.iloc[train_index, :], data.iloc[test_index, :]
        if robustse is None:
            model = smf.ols(formula, data=data_train).fit()
        else:
            model = smf.ols(formula, data=data_train).fit(cov_type=robustse)
        regression_list += [model]
        predicts_on_test += [model.predict(data_test)]
        rsquared += [model.rsquared]
        rmse_list += [rmse(data_train[formula.split("~")[0]], model.predict())]

    return {
        "regressions": regression_list,
        "test_predict": predicts_on_test,
        "r2": rsquared,
        "rmse": rmse_list,
    }


def summarize_cv(cvlist, stat="rmse"):
    result = pd.DataFrame(
        {"Model" + str(x + 1): cvlist[x][stat] for x in range(len(cv_list))}
    )
    result["Resample"] = ["Fold" + str(x + 1) for x in range(len(cvlist[0]["rmse"]))]
    result = result.set_index("Resample")
    result = pd.concat([result, pd.DataFrame(result.mean(), columns=["Average"]).T])
    return result

In [83]:
cv1 = cv_reg("wage~age", data, k, "HC0")
cv2 = cv_reg("wage~ age + age_2 + age_3 + gender", data, k, "HC0")
cv3 = cv_reg(
    "wage~ age + age_2 + age_3 + gender + edu_ba + edu_ma + edu_phd",
    data,
    k,
    "HC0")
cv4 = cv_reg(
    "wage~ age + age_2 + age_3 + gender + edu_ba + edu_ma + edu_phd + union + race",
    data,
    k,
    "HC0")
cv_list = [cv1, cv2, cv3, cv4]

In [84]:
summarize_cv(cv_list).round(2)

,Model1,Model2,Model3,Model4
Fold1,16.59,16.34,15.71,15.71
Fold2,16.05,15.89,14.85,14.82
Fold3,14.43,14.18,13.97,13.94
Fold4,16.73,16.52,15.77,15.76
Average,15.95,15.73,15.08,15.06


The cross-validated RMSE shows that Model 4 is performing the best, however Model 3 is just 0.02 higher, therefore again Model 3 would be the preferred choice considering that it is a simpler model than Model 4.